## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chara,RU,56.9064,118.2631,40.69,63,48,1.97,scattered clouds
1,1,Hovd,MN,48.0056,91.6419,54.54,17,0,2.10,clear sky
2,2,Sao Filipe,CV,14.8961,-24.4956,75.79,69,100,7.72,overcast clouds
3,3,Isangel,VU,-19.5500,169.2667,77.94,61,96,13.65,overcast clouds
4,4,Vicuna,CL,-30.0319,-70.7081,47.93,52,1,2.04,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input('Please enter minimum temperature for your destination. '))
max_temp = float(input('Please enter maximum temperature for your destination. '))

Please enter minimum temperature for your destination. 80
Please enter maximum temperature for your destination. 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

filtered_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp)&(city_data_df['Max Temp'] >= min_temp)]
                                                                       
filtered_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Butaritari,KI,3.0707,172.7902,80.82,74,18,7.45,few clouds
9,9,Balabac,PH,7.9868,117.0645,83.71,76,96,4.45,overcast clouds
12,12,Labuhan,ID,-6.8844,112.2051,80.85,85,100,5.37,overcast clouds
16,16,Denpasar,ID,-8.6500,115.2167,80.26,94,20,3.44,few clouds
41,41,Ngama,TD,11.7833,17.1667,88.14,45,28,7.96,scattered clouds
60,60,Namatanai,PG,-3.6667,152.4333,85.41,62,10,3.65,clear sky
61,61,Saint-Georges,GD,12.0564,-61.7485,80.28,83,20,20.71,few clouds
62,62,Niono,ML,14.2526,-5.9930,85.87,12,97,10.89,overcast clouds
63,63,Coahuayana,MX,18.7333,-103.6833,81.01,66,100,4.25,overcast clouds
65,65,Loa Janan,ID,-0.5830,117.0950,80.11,86,99,2.06,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.

filtered_df.count()
filtered_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = filtered_df.copy()

clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Butaritari,KI,3.0707,172.7902,80.82,74,18,7.45,few clouds
9,9,Balabac,PH,7.9868,117.0645,83.71,76,96,4.45,overcast clouds
12,12,Labuhan,ID,-6.8844,112.2051,80.85,85,100,5.37,overcast clouds
16,16,Denpasar,ID,-8.6500,115.2167,80.26,94,20,3.44,few clouds
41,41,Ngama,TD,11.7833,17.1667,88.14,45,28,7.96,scattered clouds


In [48]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Butaritari,KI,80.82,few clouds,3.0707,172.7902,
9,Balabac,PH,83.71,overcast clouds,7.9868,117.0645,
12,Labuhan,ID,80.85,overcast clouds,-6.8844,112.2051,
16,Denpasar,ID,80.26,few clouds,-8.6500,115.2167,
41,Ngama,TD,88.14,scattered clouds,11.7833,17.1667,
60,Namatanai,PG,85.41,clear sky,-3.6667,152.4333,
61,Saint-Georges,GD,80.28,few clouds,12.0564,-61.7485,
62,Niono,ML,85.87,overcast clouds,14.2526,-5.9930,
63,Coahuayana,MX,81.01,overcast clouds,18.7333,-103.6833,
65,Loa Janan,ID,80.11,overcast clouds,-0.5830,117.0950,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.

for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long = row['Lng']
    location = f"{lat},{long}"
    params['location'] = location
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_data = requests.get(url, params=params)
    hotel_data = hotel_data.json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        
        hotel_df.loc[index,'Hotel Name'] = hotel_data['results'][0]['name']
    
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [52]:
import numpy as np

In [75]:
# 7. Drop the rows where there is no Hotel Name.

keep_hotels = hotel_df['Hotel Name'] != ""

clean_hotel_df = hotel_df[keep_hotels]

clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Butaritari,KI,80.82,few clouds,3.0707,172.7902,Isles Sunset Lodge
9,Balabac,PH,83.71,overcast clouds,7.9868,117.0645,Batalla’s Lodging House
12,Labuhan,ID,80.85,overcast clouds,-6.8844,112.2051,DoubleFAG
16,Denpasar,ID,80.26,few clouds,-8.6500,115.2167,OYO 3156 Hotel Warta 2
60,Namatanai,PG,85.41,clear sky,-3.6667,152.4333,Seagulls Inn Namatani
61,Saint-Georges,GD,80.28,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
62,Niono,ML,85.87,overcast clouds,14.2526,-5.9930,Hôtel Campement
63,Coahuayana,MX,81.01,overcast clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
65,Loa Janan,ID,80.11,overcast clouds,-0.5830,117.0950,Ss
66,Kununurra,AU,84.18,clear sky,-15.7667,128.7333,Hotel Kununurra


In [78]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [88]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [92]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))